# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import ClassifierChain
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

import sqlite3 
pd.set_option('display.max_column', 999)
pd.set_option('display.max_row', 999)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings


from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load data 
First step is to establish engine and connection to sqlite to be able to upload the data from process_data.py.

Data is read into a dataframe by the load_data function which returns X, y and categories (the category columns )

In [5]:
engine = create_engine('sqlite:///Database.db')

In [6]:
conn = sqlite3.connect('Database.db')

In [7]:
Data=pd.read_sql('SELECT * FROM Data', con = conn)

In [8]:
engine = create_engine('sqlite:///Database.db')
conn = engine.connect()
df = pd.read_sql("SELECT * FROM Data", con=conn)

In [46]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### Load_data 
Function for loading data from the ETL step into a dataframe

In [98]:
# load data from database
def load_data_old():
#def load_data_old(database_filepath):
    engine = create_engine('sqlite:///Database.db')
    conn = sqlite3.connect('Database.db')
    pd.read_sql('SELECT * FROM Data', con = conn).head()
    df=pd.read_sql('SELECT * FROM Data', con = conn)
    X = df.message.values
    y = df[df.columns[4:]]
    category_names = list(df.columns)
       
    return df, X, y, category_names

In [9]:
def load_data():
    engine = create_engine('sqlite:///Database.db')
    conn = engine.connect()
    df = pd.read_sql("SELECT * FROM Data", con=conn)
    X = df['message']
    y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
    #category_names = list(df.columns)
    category_names = list(y.columns)
    return df, X, y, category_names

In [10]:
df, X, y, category_names = load_data()

In [47]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [11]:
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22621 entries, 0 to 22620
Data columns (total 40 columns):
id                        22621 non-null int64
message                   22621 non-null object
original                  10153 non-null object
genre                     22621 non-null object
related                   22621 non-null int64
request                   22621 non-null int64
offer                     22621 non-null int64
aid_related               22621 non-null int64
medical_help              22621 non-null int64
medical_products          22621 non-null int64
search_and_rescue         22621 non-null int64
security                  22621 non-null int64
military                  22621 non-null int64
child_alone               22621 non-null int64
water                     22621 non-null int64
food                      22621 non-null int64
shelter                   22621 non-null int64
clothing                  22621 non-null int64
money                     22621 non-null i

In [51]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

### Tokenization
The function 'tokenize' will first do a basic clean of the message text strings.
The next steps is tokenizing and lemmatizing, performed by imported methods. The functiom returns clean tokens.

In [42]:
def tokenize(text):

    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()    

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
   #optional:stemmer    
   # stemmer = PorterStemmer()
   # stop_words = stopwords.words("english")
   # stemmed = [stemmer.stem(word) for word in clean_tokens if word not in stop_words]
   #return stemmed

   return clean_tokens

In [52]:
tokenize(df.message[10])

['noth', 'eat', 'water', 'starv', 'thirsti']

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

#### Build the model
Function 'build_model' builds the pipeline and model, with a set of parameters and a GridSearch

In [43]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])    

In [44]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

#### Make test and train sets 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [45]:
np.random.seed(9)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [19]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i])
        recall = recall_score(actual[:, i], predicted[:, i])
        f1 = f1_score(actual[:, i], predicted[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df

In [53]:
# Calculate evaluation metrics for training set
y_train_pred = pipeline.predict(X_train)
col_names = list(y.columns.values)

print(get_eval_metrics(np.array(y_train), y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.984266   0.983719  0.995782  0.989714
request                 0.974825   0.994380  0.857984  0.921160
offer                   0.998575   1.000000  0.692308  0.818182
aid_related             0.970194   0.994973  0.933000  0.962990
medical_help            0.979694   0.997998  0.745699  0.853596
medical_products        0.986166   1.000000  0.728438  0.842886
search_and_rescue       0.991984   0.993671  0.702461  0.823067
security                0.995309   0.995349  0.732877  0.844181
military                0.990381   0.995238  0.723183  0.837675
water                   0.983850   0.997558  0.751610  0.857293
food                    0.975656   0.995924  0.783957  0.877319
shelter                 0.978803   0.998253  0.763017  0.864926
clothing                0.995962   0.988827  0.728395  0.838863
money                   0.994359   0.996255  0.738889  0.848485
missing_people          0.996972   1.000

In [54]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    
    parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}    
   
    cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 2)

    return cv

In [ ]:
#previous parameter tryouts
#  parameters = {
  #      'vect__ngram_range': ((1, 1), (1, 2)),
  #      'clf__estimator__min_samples_split': [2, ],
  #      'vect__max_df': (0.5, 0.75, 1.0)
   # }
   # parameters = {
   #     'vect__ngram_range': ((1, 1), (1, 2)),
   #     'vect__max_df': (0.5, 0.75, 1.0),
   #     'vect__max_features': (None, 5000, 10000),
   #     'tfidf__use_idf': (True, False),
   # }
   # cv = GridSearchCV(pipeline, param_grid=parameters)

#### Building the model and assessing the results

In [55]:
model = build_model()
model

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__min_df': [1, 5], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

#### Build model - first version
Below is the first pipeline build, before adding MultiOutput and Grid search

In [49]:
def build_model_old():   
  
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])
    
    return pipeline    

In [50]:
model_old = build_model_old()

In [51]:
model_old

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### Splitting the data
Train and test sets are established by using the train_test_split method, imported from sklearn models

#### Train and test datasets
'Train_test_split' is used to establish the train and test datasets for X and y

In [21]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
np.random.seed(9)

### Fitting the model
The model from build_data is fitted to the train datasets.
This step is very time-consuming, taking ca 1 full hour to complete for this version 

In [54]:
model_old.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [77]:
model.fit(X_train, y_train)

#### Evaluation metrics
Below is evaluation metrics. As I was struggling to debug my initial metrics (finally found I had to specify ''.values').

I am (grateful for finding this setup on GitHub. I managed to fix my first metrics eventually, but kept both in the Notebook.

In [56]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i])
        recall = recall_score(actual[:, i], predicted[:, i])
        f1 = f1_score(actual[:, i], predicted[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df

In [57]:
# Calculation the proportion of each column that have label == 1
y.sum()/len(y)

related                   0.759619
request                   0.173762
offer                     0.004409
aid_related               0.417038
medical_help              0.079934
medical_products          0.050677
search_and_rescue         0.027031
security                  0.018213
military                  0.034378
water                     0.065194
food                      0.111997
shelter                   0.090221
clothing                  0.014918
money                     0.022533
missing_people            0.010777
refugees                  0.033443
death                     0.044621
other_aid                 0.131902
infrastructure_related    0.063903
transport                 0.046090
buildings                 0.051256
electricity               0.020440
tools                     0.006234
hospitals                 0.010509
shops                     0.004409
aid_centers               0.011934
other_infrastructure      0.043240
weather_related           0.269861
floods              

In [58]:
# Calculate evaluation metrics for training set
y_train_pred = pipeline.predict(X_train)
col_names = list(y.columns.values)

print(get_eval_metrics(np.array(y_train), y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.984266   0.983719  0.995782  0.989714
request                 0.974825   0.994380  0.857984  0.921160
offer                   0.998575   1.000000  0.692308  0.818182
aid_related             0.970194   0.994973  0.933000  0.962990
medical_help            0.979694   0.997998  0.745699  0.853596
medical_products        0.986166   1.000000  0.728438  0.842886
search_and_rescue       0.991984   0.993671  0.702461  0.823067
security                0.995309   0.995349  0.732877  0.844181
military                0.990381   0.995238  0.723183  0.837675
water                   0.983850   0.997558  0.751610  0.857293
food                    0.975656   0.995924  0.783957  0.877319
shelter                 0.978803   0.998253  0.763017  0.864926
clothing                0.995962   0.988827  0.728395  0.838863
money                   0.994359   0.996255  0.738889  0.848485
missing_people          0.996972   1.000

In [27]:
# Calculate evaluation metrics for training set
y_train_pred = model.predict(X_train)
col_names = list(y.columns.values)

print(get_eval_metrics(np.array(y_train), y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.959268   0.949677  0.999375  0.973893
request                 0.965622   0.999135  0.800139  0.888632
offer                   0.996081   1.000000  0.153846  0.266667
aid_related             0.986759   0.997504  0.970571  0.983853
medical_help            0.965028   0.997340  0.560957  0.718047
medical_products        0.974944   0.997717  0.509324  0.674383
search_and_rescue       0.984325   1.000000  0.409396  0.580952
security                0.988303   1.000000  0.325342  0.490956
military                0.982128   0.996416  0.480969  0.648775
water                   0.967640   1.000000  0.498620  0.665439
food                    0.951550   0.999053  0.564171  0.721121
shelter                 0.962059   0.998839  0.574099  0.729123
clothing                0.988897   1.000000  0.230453  0.374582
money                   0.986166   1.000000  0.352778  0.521561
missing_people          0.991866   1.000

In [159]:
# Calculate evaluation metrics for training set
y_test_pred = pipeline.predict(X_test)
col_names = list(y.columns.values)

print(get_eval_metrics(np.array(y_train), y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.988778   0.986952  0.998438  0.992662
request                 0.978981   0.997252  0.879806  0.934855
offer                   0.998575   1.000000  0.692308  0.818182
aid_related             0.970372   0.994975  0.933429  0.963220
medical_help            0.979100   0.998987  0.737472  0.848537
medical_products        0.986819   1.000000  0.741259  0.851406
search_and_rescue       0.992341   1.000000  0.711409  0.831373
security                0.995309   0.995349  0.732877  0.844181
military                0.990856   0.997653  0.735294  0.846614
water                   0.984384   1.000000  0.758050  0.862376
food                    0.975656   0.998634  0.781818  0.877025
shelter                 0.980287   0.998291  0.779706  0.875562
clothing                0.996497   0.994624  0.761317  0.862471
money                   0.994241   1.000000  0.730556  0.844302
missing_people          0.997031   1.000

#### Testing the simple standalone predict and looking at the results 

In [109]:
y_pred = pipeline.predict(X_test)

In [31]:
y_pred = model.predict(X_test)

In [43]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

#### Display result
The display_result function was used for the model before MultiOutputClassifier was added 
It cannot be used for MultiOutput

In [50]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [ ]:
display_results(y_test, y_pred)  

#### Dataframe versions of y_test and y_pred
Need to make these sets dataframes to be usen in the evaluation function

In [32]:
y_test_df= pd.DataFrame(data=y_test)
y_pred_df= pd.DataFrame(data=y_pred)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
print("Accuracy scores for each category\n")
print("*-" * 30)

for i in range(35):
    print("Category:", category_names[i],"\n", classification_report(y_test_df.values[:, i], y_pred_df.values[:, i]))

Accuracy scores for each category

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Category: related 
              precision    recall  f1-score   support

          0       0.38      0.02      0.04      1359
          1       0.76      0.99      0.86      4255

avg / total       0.67      0.75      0.66      5614

Category: request 
              precision    recall  f1-score   support

          0       0.82      1.00      0.90      4599
          1       0.44      0.01      0.02      1015

avg / total       0.75      0.82      0.74      5614

Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5593
          1       0.00      0.00      0.00        21

avg / total       0.99      1.00      0.99      5614

Category: aid_related 
              precision    recall  f1-score   support

          0       0.59      0.86      0.70      3249
          1       0.47      0.17      0.25      2365

avg / total     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [138]:
def evaluate_model(model, X_test, y_test, category_names):
    """
    Evaluates the trained model by predicting on test data
    The classification report prints output accuracy, precision and recall for all categories
    """
    y_pred = model.predict(X_test)
    for i in range(0, len(category_names)):
        print("Category:", category_names[i])
        print(classification_report(y_test_df.values[:,i], y_pred_df.values[:,i]))      
        

In [22]:
#evaluate_model(model_old, X_test, y_test, category_names)

In [76]:
#y_train_pred = model.predict(X_train)
#col_names = list(y.columns.values)

#print(get_eval_metrics(np.array(y_train), y_train_pred, col_names))

### 6. Improve your model
Use grid search to find better parameters. 

### Grid search was introduced in the pipeline and tested. 
The code is updated and the latest version is in above cells

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [165]:
#y_pred = model.predict(X_test)
for i in range(0, len(category_names)):
        print("Category:", category_names[i])
        print("'Accuracy: ", accuracy_score(y_test_df.values[:,i], y_pred_df.values[:,i]))
        print(classification_report(y_test_df.values[:,i], y_pred_df.values[:,i]))

Category: id
'Accuracy:  0.754542215889
             precision    recall  f1-score   support

          0       0.38      0.02      0.04      1359
          1       0.76      0.99      0.86      4255

avg / total       0.67      0.75      0.66      5614

Category: message
'Accuracy:  0.818667616673
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      4599
          1       0.44      0.01      0.02      1015

avg / total       0.75      0.82      0.74      5614

Category: original
'Accuracy:  0.996259351621
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5593
          1       0.00      0.00      0.00        21

avg / total       0.99      1.00      0.99      5614

Category: genre
'Accuracy:  0.570894193089
             precision    recall  f1-score   support

          0       0.59      0.86      0.70      3249
          1       0.47      0.17      0.25      2365

avg / total       0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


IndexError: index 35 is out of bounds for axis 1 with size 35

In [23]:
#y_pred_old = model_old.predict(X_test)
#print('Accuracy:', accuracy_score(y_test, y_pred_old))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

#### Saving model as pickle file - 'Model'

In [68]:
def save_model1(model, model_filepath):
    file_Name = 'model_filepath'   
    fileObject = open(file_Name,'wb')    
    pickle.dump(model,fileObject)    
    fileObject.close()    

In [78]:
save_model1(model, 'mymodel.pkl')

In [17]:
def save_model2(model, model_filepath):
    pickle.dump(model, open(model_filepath, "wb"))  
    print('model saved in:', model_filepath)

In [18]:
save_model2(model, 'modelnew2.pkl')

model saved in: modelnew2.pkl


In [15]:
def save_model3(model, model_filepath):
    joblib.dump(model, model_filepath)
    print('joblib model saved in:', model_filepath)
    joblib.dump(model, 'joblib_model.pkl')
    print('joblib model saved in:', 'joblib_model.pkl')

In [16]:
save_model3(model, 'modelnew.pkl')

joblib model saved in: modelnew.pkl
joblib model saved in: joblib_model.pkl


In [71]:
pkl_filename = "pickle_model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(model, file)

In [86]:
# Load from file
with open(pkl_filename, 'rb') as file:  
    pickle_model = pickle.load(file)

In [87]:
pickle_model

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__min_samples_split': [2, 4], 'vect__max_df': (0.5, 0.75, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
score = pipeline.score(X_test, y_test)  
print("Test score: {0:.2f} %".format(100 * score)) 

Test score: 14.61 %


In [89]:
score = pickle_model.score(X_test, y_test)  
print("Test score: {0:.2f} %".format(100 * score))  
#Ypredict = pickle_model.predict(Xtest)  

Test score: 14.67 %


In [34]:
score = model.score(X_test, y_test)  
print("Test score: {0:.2f} %".format(100 * score)) 

Test score: 15.60 %


In [72]:
from sklearn.externals import joblib
# Save to file in the current working directory
joblib_file = "joblib_model.pkl"  
joblib.dump(model, joblib_file)

['joblib_model.pkl']

In [74]:
# Load from file
#joblib_model = joblib.load(joblib_file)

In [75]:
#model=joblib.load(newmodel.pkl)

In [82]:
joblib_model

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__min_samples_split': [2, 4], 'vect__max_df': (0.5, 0.75, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [152]:
# Calculate the accuracy and predictions
score = model.score(X_test, y_test)  
print("Test score: {0:.2f} %".format(100 * score)) 

Test score: 15.60 %


In [90]:
Ypredict = joblib_model.predict(X_test)  

In [91]:
Ypredict

array([['1', '0', '0', ..., '0', '0', '0'],
       ['1', '0', '0', ..., '0', '0', '0'],
       ['1', '0', '0', ..., '0', '0', '0'],
       ..., 
       ['1', '0', '0', ..., '0', '0', '0'],
       ['1', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0']], dtype=object)

In [73]:
#evaluate_model(joblib_model, X_test, y_test, category_names)

### Loading model from pickle file 

In [63]:
def load_model(model_file):
# we open the file for reading
    fileObject = open('mymodel','rb')  
    # load the object from the file into var modelX
    model = pickle.load(fileObject)  
    model

In [12]:
from sklearn.externals import joblib

In [134]:
filename = 'newmodel.pkl'
joblib.dump(model, filename)

['newmodel.pkl']

In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
joblib.dump(model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)

In [ ]:
#Use the load below

In [71]:
model = joblib.load('newmodel.pkl')

In [72]:
model

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__min_df': [1, 5], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

#### Putting all steps together
Below is a main function with the steps needed to run the code, calling the functions

In [ ]:
def main():
    df, X, y, category_names = load_data()
    clean_tokens= tokenize(text)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    get_eval_metrics(actual, predicted, col_names):
    evaluate_model(model, y_test, y_pred)
    save_model(model, model_filepath)
    

In [ ]:
main()

#### Testing the main program stucture with main()

In [ ]:
Testing visuals

In [142]:
genre_counts = df.groupby('genre').count()['message']
genre_names = list(genre_counts.index)
genre_counts

genre
direct    10646
news       9468
social     2342
Name: message, dtype: int64

In [147]:
    # Show distribution of different category
category = list(df.columns[4:])
category_counts = []
    for column_name in category:
        category_counts.append(np.sum(df[column_name]))

    # extract data exclude related
categories = df.iloc[:,4:]
categories_mean = categories.mean().sort_values(ascending=False)[1:11]
categories_names = list(categories_mean.index)


In [148]:
category_counts

[17058,
 3902,
 99,
 9365,
 1795,
 1138,
 607,
 409,
 772,
 1464,
 2515,
 2026,
 335,
 506,
 242,
 751,
 1002,
 2962,
 1435,
 1035,
 1151,
 459,
 140,
 236,
 99,
 268,
 971,
 6060,
 1794,
 1961,
 228,
 2056,
 426,
 1145,
 4385]

In [27]:
catcount_df = pd.DataFrame(catcounts)

In [38]:
categories = df.iloc[:, 4:]
cat_names = list(categories)
catcounts = [df[cat_name].value_counts for cat_name in cat_names]
#catcount = categories.apply(lambda x: x.value_counts()).T.stack()
#catcounts

In [40]:
#catcount_df.head()
#catcount_df.describe

In [41]:
category = list(df.columns[4:])
category_counts = []
for column_name in category:
    category_counts.append(np.sum(df[column_name]))

In [29]:
#category_counts

In [32]:
categories = df.iloc[:,4:]
categories_mean = categories.mean().sort_values(ascending=False)[1:15]
categories_names = list(categories_mean.index)


In [33]:
categories_mean

aid_related               0.417038
weather_related           0.269861
direct_report             0.195271
request                   0.173762
other_aid                 0.131902
food                      0.111997
earthquake                0.091557
shelter                   0.090221
storm                     0.087326
medical_help              0.079934
floods                    0.079890
water                     0.065194
infrastructure_related    0.063903
buildings                 0.051256
dtype: float64

In [79]:
category = list(df.columns[4:])
for column_name in category:
    df[[column_name]] = df[[column_name]].apply(pd.to_numeric)

In [76]:
df[['related']] = df[['related']].apply(pd.to_numeric)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22621 entries, 0 to 22620
Data columns (total 34 columns):
request                   22621 non-null int64
offer                     22621 non-null int64
aid_related               22621 non-null int64
medical_help              22621 non-null int64
medical_products          22621 non-null int64
search_and_rescue         22621 non-null int64
security                  22621 non-null int64
military                  22621 non-null int64
water                     22621 non-null int64
food                      22621 non-null int64
shelter                   22621 non-null int64
clothing                  22621 non-null int64
money                     22621 non-null int64
missing_people            22621 non-null int64
refugees                  22621 non-null int64
death                     22621 non-null int64
other_aid                 22621 non-null int64
infrastructure_related    22621 non-null int64
transport                 22621 non-null int6

In [82]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [83]:
genre_counts

genre
direct    10747
news       9480
social     2394
Name: message, dtype: int64

In [44]:
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [42]:
#counts
#cat_names

In [35]:
genre_counts = df.groupby('genre').count()['message']
genre_names = list(genre_counts.index)
    
categories = df.iloc[:, 4:]
cat_names = list(categories)
cat_counts = [df[cat_name].sum() for cat_name in cat_names]
    
categories['sum'] = categories.sum(axis=1)
counts = categories.groupby('sum').count()['related']
names = list(counts.index)

In [45]:
#plot(graphs3, validate=False)